In [13]:
# use pandas to load data and visualize
# use longformer tokenizer (efficient at tokenizing large texts)
# Set Up the Model for Fine-Tuning (transition to Colab here?)
# Define Training Arguments

In [35]:
# Visualizing the data with pandasimport pandas as pd
import pandas as pd
import json

data = pd.read_json('./datasets/subtaskA_dev_monolingual.jsonl', lines=True)
print(data.head())

                                                text  label   model   source  \
0  Giving gifts should always be enjoyable.  Howe...      1  bloomz  wikihow   
1  Yveltal (Japanese: ユベルタル) is one of the main a...      1  bloomz  wikihow   
2  If you'd rather not annoy others by being rude...      1  bloomz  wikihow   
3  If you're interested in visiting gravesite(s) ...      1  bloomz  wikihow   
4  The following are some tips for becoming succe...      1  bloomz  wikihow   

   id  
0   0  
1   1  
2   2  
3   3  
4   4  


In [36]:
# tokenizer function
from transformers import LongformerTokenizer


tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
# #print(tokenizer("Hello world")["input_ids"])
# print(tokenizer('./datasets/subtaskA_dev_monlingual.jsonl')["input_ids"])
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=4096)

In [38]:
from datasets import Dataset

dataset = Dataset.from_pandas(data)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
print(tokenized_dataset)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:06<00:00, 731.99 examples/s]

Dataset({
    features: ['text', 'label', 'model', 'source', 'id', 'input_ids', 'attention_mask'],
    num_rows: 5000
})


TypeError: string indices must be integers, not 'str'